# Simulated Annealing on MaxCut Problem

In this notebook, we demonstrate Simulated Annealing on the MaxCut problem. 

Simulated annealing is an optimization algorithm inspired by the annealing process in metallurgy. It explores the solution space by accepting both better and worse neighboring states with a probability that decreases over time. At high temperatures, the algorithm explores broadly and avoids local optima. As the temperature lowers, it refines the solution and converges to an optimal or near-optimal state.

1. [Introduction](#intro)
2. [Temperature Decay](#tempdecay)
3. [Simulated Annealing Initialization](#siminit)
4. [Testing](#testing)

<a id='intro'></a>
## 1. Introduction

### Imports

In [ ]:
%pip install numpy
%pip install networkx
%pip install torch
%pip install pandas

In [1]:
import copy
import time
from typing import List, Union
import numpy as np
import random
import networkx as nx
from util import read_nxgraph
from util import obj_maxcut

copy: deep copies <br>
time: measurement <br>
typing: type annotations <br>
numpy: arrays<br>
random: sampling <br>
networkx: graph manipulation/analysis <br>
util: read graphs, calculate Max-Cut objective function

<a id='tempdecay'></a>
## 2. Temperature Decay Function

This loop features the temperature decay and comparison of results for further traversal in the algorithm.

The temperature decay function is given by:

$$ temp = initTemp * (\frac{1-(currStep+1)}{numSteps})$$

The probability calculation used to determine if the algorithm should continue exploring other parts of the function is given by:

$$ \mathbb{P} = e^{\frac{-(currScore - newScore)}{(temp + 1^{-6})}}$$

In [2]:
def temperature_decay(num_steps, graph):
    """
    Simulates the temperature decay process during simulated annealing for solving the Max-Cut problem.

    Parameters:
    -- num_steps: Number of steps
    -- graph: A NetworkX graph for the MaxCut problem

    Return:
    -- scores: List of objective values at each step
    -- init_score: Initial objective value of solution
    -- curr_score: Final objective value after simulated annealing
    -- curr_solution: Final solution partitioning of nodes
    """
    init_solution = [0] * int(graph.number_of_nodes() / 2) + [1] * int(graph.number_of_nodes() / 2)
    curr_solution = copy.deepcopy(init_solution)
    # Gives us the maximum cut value (objective value) of current partitioning
    curr_score = obj_maxcut(curr_solution, graph)
    init_score = curr_score
    num_nodes = len(init_solution)
    scores = []
    for k in range(num_steps):
        # The temperature decreases
        temperature = init_temperature * (1 - (k + 1) / num_steps)
        new_solution = copy.deepcopy(curr_solution)
        idx = np.random.randint(0, num_nodes)
        new_solution[idx] = (new_solution[idx] + 1) % 2
        new_score = obj_maxcut(new_solution, graph)
        scores.append(new_score)
        delta_e = curr_score - new_score
        if delta_e < 0:
            curr_solution = new_solution
            curr_score = new_score
        else:
            prob = np.exp(- delta_e / (temperature + 1e-6))
            if prob > random.random():
                curr_solution = new_solution
                curr_score = new_score

    return scores, init_score, curr_score, curr_solution

<a id='siminit'></a>
## 3. Simulated Annealing Initialization

In [3]:
def simulated_annealing(init_temperature: int, num_steps: int, graph: nx.Graph) -> (int, Union[List[int], np.array], List[int]):
    """
    Performs simulated annealing process for Max Cut.

    Parameters:
    -- init_temperature: Starting temperature for simulated annealing
    -- num_steps: Number of steps
    -- graph: A NetworkX graph for the MaxCut problem

    Return:
    -- curr_score: Final objective value after simulated annealing
    -- curr_solution: Final solution partitioning of nodes
    -- scores: List of objective values at each step
    """
    print('simulated_annealing')

    start_time = time.time()

    scores, init_score, curr_score, curr_solution = temperature_decay(num_steps, graph)
    
    print("score, init_score of simulated_annealing:", curr_score, init_score)
    print("solution: ", curr_solution)
    running_duration = time.time() - start_time
    print('running_duration: ', running_duration)
    return curr_score, curr_solution, scores

<a id='testing'></a>
## 4. Testing

Below are descriptions of the graphs in our dataset (sourced from https://web.stanford.edu/~yyye/yyye/Gset/). 
| Graph | # Nodes | # Edges |
|-------|---------|---------|
|  G14  |   800   |   4694  |
|  G15  |   800   |   4661  |
|  G22  |  2000   |  19990  |
|  G49  |  3000   |   6000  |
|  G50  |  3000   |   6000  |
|  G55  |  5000   |  12468  |
|  G70  | 10000   |   9999  |

Run the following to get the results of simulated annealing on G14.

In [4]:
if __name__ == '__main__':
    graph = read_nxgraph('./data/gset_14.txt')
    init_temperature = 4
    num_steps = 2000
    sa_score, sa_solution, sa_scores = simulated_annealing(init_temperature, num_steps, graph)

simulated_annealing
score, init_score of simulated_annealing: 2791.0 1934.0
solution:  [0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 

<a id='benchmark'></a>
### 4.1 Benchmarked Results

These results demonstrate the score given to the Simulated Annealing Algorithm on 7 benchmarked graphs given in data. Additionally, a graph corresponding to the time taken to complete each test is provided. <br>
![title](images/simulated_annealing_scores.png)